In [3]:
from google.colab import drive
drive.mount("/content/drive")
%cd drive/MyDrive/Colab\ Notebooks/EandI

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/EandI


In [4]:
import sys
sys.path.append("..")
from models.resnet import ResNet
from utils.data import LitDataloader
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import optuna
from optuna.integration import PyTorchLightningPruningCallback
from pl_bolts.callbacks import PrintTableMetricsCallback
from sklearn.model_selection import StratifiedKFold
import numpy as np
from datetime import datetime
import torch
import pickle

In [ ]:
def objective(trial):
    '''
    n_blocks = trial.suggest_int("n_blocks", 2, 3)
    if n_blocks == 2:
      feature_maps = trial.suggest_categorical("feature_maps", [[32, 64], [64, 128], [64, 64], [128, 64]])
    elif n_blocks == 3:
      feature_maps = trial.suggest_categorical("feature_maps", [[32, 64, 128], [64, 128, 128], [64, 64, 128]])
    elif n_blocks == 4:
      feature_maps = trial.suggest_categorical("feature_maps", [[32, 64, 128], [64, 128, 128], [64, 64, 128]])
    '''

    n_blocks = 3
    #map_key = trial.suggest_categorical("feature_maps", ['1', '2', '3', '4', '5', '6', '7', '8'])
    #feature_maps = {'1': [32, 64, 128], '2': [64, 128, 128], '3': [64, 64, 128], '4':[128, 64, 32], 
    #                '5': [64, 64], '6': [32, 64], '7': [32, 32], '8': [16, 32]}
    #feature_map = feature_maps[map_key]
    feature_map = [64, 128, 128]
    input_dim = 1725
    
    activation = 'relu'
    optimizer = 'Adam'
    lr = trial.suggest_loguniform('learning_rate', 1e-8, 0.01)
    #weight_decay = trial.suggest_loguniform('weight_decay', 1e-8, 0.001)
    dropout = trial.suggest_float('dropout', low=0.0, high=0.5, step=0.1)
    #batchnorm = trial.suggest_categorical('batchnorm', [False])
    #print('Batchnorm: {}'.format(batchnorm))
    print('learning rate: {}'.format(lr))
    #print('trial evaluating feature maps: {}'.format(feature_map))

    resnet = ResNet(num_channels=1, num_classes=2, 
                    num_blocks=len(feature_map), 
                    feature_maps=feature_map, 
                    batchnorm=False,
                    activation=activation, 
                    optimizer=optimizer, 
                    lr=lr, 
                    dropout=dropout,
                    class_weights=[0.3, 7.0])
    print(resnet)
    DataLoader = LitDataloader('data/Haleh/', batch_size=128, imbalanced_sampler=True)
    train, val = DataLoader.train_dataloader(), DataLoader.val_dataloader()
    logger = TensorBoardLogger('tb_logs', 'ResNet_Tune')
    trainer = pl.Trainer(gpus=1,
                         max_epochs=20,
                         auto_lr_find=True,
                         logger=logger,
                         callbacks=[PyTorchLightningPruningCallback(trial, monitor='val_loss'), 
                                    pl.callbacks.EarlyStopping('val_auroc', patience=5)]
                         )
    trainer.fit(resnet, train, val)

    score = trainer.test(resnet, val)
    f1 = score[0]['test_f1']
    auroc = score[0]['test_auroc']
    precision = score[0]['test_precision']
    recall = score[0]['test_recall']
    print(f'AUROC: {auroc} \t F1: {f1} \t Precision: {precision} \t Recall: {recall}')

    return f1

In [ ]:
study_name = 'ResNet'
pruner=optuna.pruners.MedianPruner(n_warmup_steps=5)
study = optuna.create_study(pruner=pruner,
                            direction='maximize',
                            study_name=study_name,
                            storage='sqlite:///optuna_resnet.db',
                            load_if_exists=True)
study.optimize(objective, n_trials=20)
print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))

[I 2022-01-07 14:03:42,066] Using an existing study with name 'ResNet' instead of creating a new one.
/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning:

Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.



learning rate: 1.830983938630999e-05
ResNet(
  (activate): ReLU()
  (loss): BCEWithLogitsLoss()
  (train_metrics): MetricCollection(
    (AUROC): AUROC()
    (Accuracy): Accuracy()
    (F1): F1()
    (Precision): Precision()
    (Recall): Recall(),
    prefix=train_
  )
  (val_metrics): MetricCollection(
    (AUROC): AUROC()
    (Accuracy): Accuracy()
    (F1): F1()
    (Precision): Precision()
    (Recall): Recall(),
    prefix=val_
  )
  (test_metrics): MetricCollection(
    (AUROC): AUROC()
    (Accuracy): Accuracy()
    (F1): F1()
    (Precision): Precision()
    (Recall): Recall(),
    prefix=test_
  )
  (RESIDUAL_BLOCKS): Sequential(
    (BLOCK_#1): ResidualBlock(
      (activate): ReLU()
      (conv_part): Sequential(
        (Conv1D_1): Conv1d(1, 64, kernel_size=(7,), stride=(1,), padding=(3,), padding_mode=reflect)
        (activation_1): ReLU()
        (Conv1D_2): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), padding_mode=reflect)
        (activation_2): ReLU()


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type              | Params
------------------------------------------------------
0 | activate        | ReLU              | 0     
1 | loss            | BCEWithLogitsLoss | 0     
2 | train_metrics   | MetricCollection  | 0     
3 | val_metrics     | MetricCollection  | 0     
4 | test_metrics    | MetricCollection  | 0     
5 | RESIDUAL_BLOCKS | Sequential        | 477 K 
6 | FINAL_LAYER     | Sequential        | 32.8 K
7 | softmax         | Softmax           | 0     
------------------------------------------------------
510 K     Trainable params
0         Non-trainable params
510 K     Total params
2.041     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:658: UserWarning:

Your `val_dataloader` has `shuffle=True`, it is strongly recommended that you turn this off for val/test/predict dataloaders.

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:429: UserWarning:

The number of training samples (24) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.



...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

[I 2022-01-07 14:14:30,140] Trial 50 pruned. Trial was pruned at epoch 5.


Best trial: score 0.4639536440372467, params {'dropout': 0.5, 'learning_rate': 3.0954352203123486e-05}


In [ ]:
best_params = {'batchnorm': True, 'learning_rate': 4.430065921544051e-05} #Best Trial 0.673 (auroc+f1)/2

In [5]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")

data_path = 'data/Haleh/'
n_folds = 5

n_blocks = 3
#map_key = trial.suggest_categorical("feature_maps", ['1', '2', '3', '4'])
feature_maps = {'1': [32, 64, 128], '2': [64, 128, 128], '3': [64, 64, 128], '4':[128, 64, 32]}
#feature_map = feature_maps[map_key]
input_dim = 1725

epochs = 15

#print('trial evaluating feature maps: {}'.format(feature_map))

X = np.load(data_path + 'X_train.npy')
y = np.load(data_path + 'y_train.npy')
kfold = StratifiedKFold(n_splits=n_folds)

cv_results = {'f1': [],
              'roc': [],
              'pr': [],
              'rc': []
              }

params = {
    'n_blocks': 3,
    'activation': 'relu',
    'optimizer': 'AdamW',
    'class_weights': [0.15, 6.0],
    'weight_decay': 0.00004,
    'dropout': 0.5,
    'lr': 3.0954352203123486e-05
}

best_score = 0.0
for fold, (train_idx, val_idx) in enumerate(kfold.split(X, y)):

    resnet = ResNet(num_channels=1, 
                num_classes=2, 
                num_blocks =    params['n_blocks'], 
                activation=     params['activation'], 
                optimizer=      params['optimizer'],
                class_weights=  params['class_weights'],
                weight_decay=   params['weight_decay'],
                dropout =       params['dropout'],
                lr =            params['lr'])
    print(resnet)
    break
    trainer = pl.Trainer(gpus=1,
                     max_epochs=epochs,
                     auto_lr_find=True
                     )
    
    DataLoader = LitDataloader(data_path, batch_size=128, indices=[train_idx, val_idx], imbalanced_sampler=True, normalize=True)
    train, val = DataLoader.train_dataloader(), DataLoader.val_dataloader()
    trainer.fit(resnet, train, val)

    DataLoader = LitDataloader(data_path, batch_size=128, imbalanced_sampler=False, mode='test', normalize=True)
    test = DataLoader.test_dataloader()
    score = trainer.test(resnet, test)
    f1 = score[0]['test_f1']
    auroc = score[0]['test_auroc']
    pr = score[0]['test_precision']
    rc = score[0]['test_recall']
    cv_results['f1'].append(f1)
    cv_results['roc'].append(auroc)
    cv_results['pr'].append(pr)
    cv_results['rc'].append(rc)

    pred = trainer.predict(resnet, test)
    pred = torch.cat(pred)
    y_pred = np.argmax(pred.cpu().detach().numpy(), axis=1)
    y_proba = pred.cpu().detach().numpy()
    np.save(f'predictions/ResNet/{current_time}_y_pred_fold_{fold+1}.npy', y_pred)
    np.save(f'predictions/ResNet/{current_time}_y_proba_Fold_{fold+1}.npy', y_proba)

print(cv_results)

with open(f'predictions/ResNet/{current_time}_params.pickle', 'wb') as handle:
    pickle.dump(params, handle, protocol=pickle.HIGHEST_PROTOCOL)

/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning:

Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.



ResNet(
  (activate): ReLU()
  (loss): BCEWithLogitsLoss()
  (train_metrics): MetricCollection(
    (AUROC): AUROC()
    (Accuracy): Accuracy()
    (F1): F1()
    (Precision): Precision()
    (Recall): Recall(),
    prefix=train_
  )
  (val_metrics): MetricCollection(
    (AUROC): AUROC()
    (Accuracy): Accuracy()
    (F1): F1()
    (Precision): Precision()
    (Recall): Recall(),
    prefix=val_
  )
  (test_metrics): MetricCollection(
    (AUROC): AUROC()
    (Accuracy): Accuracy()
    (F1): F1()
    (Precision): Precision()
    (Recall): Recall(),
    prefix=test_
  )
  (RESIDUAL_BLOCKS): Sequential(
    (BLOCK_#1): ResidualBlock(
      (activate): ReLU()
      (conv_part): Sequential(
        (Conv1D_1): Conv1d(1, 64, kernel_size=(7,), stride=(1,), padding=(3,), padding_mode=reflect)
        (BatchNorm_1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation_1): ReLU()
        (Conv1D_2): Conv1d(64, 64, kernel_size=(5,), stride=(1,), p